In [1]:
import os, sys
sys.path.append("..")

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torchvision.utils import make_grid

from src.model import Glow 
from src.train import calc_z_shapes

import gc

from src.langevin_dynamics import (
    tensor_list_to_vector, vector_to_tensor_list, sample_z_list, z_list_to_image, latent_langevien_step
)

In [2]:
gc.collect(); torch.cuda.empty_cache()

### Model loading

In [3]:
N_FLOW = 32
N_BLOCK = 4
AFFINE = False
NO_LU = False
IMG_SIZE = 64
N_SAMPLE = 20
GLOW_DEVICE = "cuda:4"
DEVICE_IDS = [4]
EPS = 1

In [4]:
MODEL_SINGLE = Glow(
    3, N_FLOW, N_BLOCK, affine=AFFINE, conv_lu=not NO_LU
)
MODEL = nn.DataParallel(MODEL_SINGLE, device_ids=DEVICE_IDS)

MODEL.load_state_dict(torch.load("../data/glow_model.pt"))
SHAPES = calc_z_shapes(3, IMG_SIZE, N_FLOW, N_BLOCK)

MODEL = MODEL.to(GLOW_DEVICE)

../src/model.py:102: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  w_s = torch.from_numpy(w_s)


In [5]:
imgs = []

with torch.no_grad():
    for i in tqdm(range(500)):
        z_list = sample_z_list(batch_size=N_SAMPLE, z_shapes=SHAPES, device=GLOW_DEVICE)
        z_vector = tensor_list_to_vector(z_list)
        img = z_list_to_image(MODEL, z_list)
        
        imgs.append(img)
        
imgs_tensor = torch.cat(imgs, dim=0).cpu()

100%|██████████| 500/500 [02:46<00:00,  3.01it/s]


In [6]:
imgs_tensor = torch.cat(imgs, dim=0).cpu()

In [7]:
# torch.save(imgs_tensor + np.sqrt(EPS)*torch.randn_like(imgs_tensor),
#            f"../data/glow_generated_degrated_images_eps_{EPS}.torch")

In [8]:
torch.save(imgs_tensor,
           f"../data/glow_generated_images.torch")